Let's compare one spreadsheet to the other and see if any of the same records that exist between the 2 have changed.

In [1]:
import numpy as np
import pandas as pd

np.__version__, pd.__version__
# ('1.19.2', '1.2.2')

('1.19.2', '1.2.2')

In [2]:
def createSummary(yyyymmdd):
    
    csvName = 'data/conposcovidloc-' + yyyymmdd + '.csv'
    parseDates = ['Accurate_Episode_Date', 'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date']
    rawDf = pd.read_csv(csvName, 
                  header=0, 
                  parse_dates=parseDates)
    
    rawDf.fillna({'Test_Reported_Date':'Case_Reported_Date', 'Specimen_Date':'Case_Reported_Date', 'Outbreak_Related':'No'}, inplace=True)
    
    phuInfoColumns = ['Reporting_PHU', 'Reporting_PHU_Address','Reporting_PHU_City','Reporting_PHU_Postal_Code','Reporting_PHU_Website','Reporting_PHU_Latitude','Reporting_PHU_Longitude']
    rawDf.drop(phuInfoColumns, axis = 1, inplace=True)
    
    caseDate = 'Accurate_Episode_Date'
    rawDf['Case_Year'] = rawDf[caseDate].dt.isocalendar().year
    rawDf['Case_WeekNo'] = rawDf[caseDate].dt.isocalendar().week
    
    # groupByColumnNames = ['Client_Gender', 'Age_Group', 'Outcome1', 'Case_Year', 'Case_WeekNo']
    groupByColumnNames = ['Client_Gender', 'Age_Group', 'Case_Year', 'Case_WeekNo']
    summaryDf = rawDf.groupby(groupByColumnNames)[[caseDate]].count().reset_index()
    summaryDf.rename(columns = {caseDate:'Case_Count'}, inplace = True) 
    summaryDf['FileDate'] = yyyymmdd
    
    return summaryDf
    

In [3]:
dataFiles = ['2021-02-18','2021-02-19','2021-02-22', '2021-02-23', '2021-03-07', '2021-03-08', '2021-03-09']

In [4]:
for fileName in enumerate(dataFiles):
    
    yyyymmdd = fileName[1]
    
    if fileName[0] == 0:
        print('create ', yyyymmdd)
        allDaysDf = createSummary(yyyymmdd)
    else:
        print('append ', yyyymmdd)
        thisDf = createSummary(yyyymmdd)
        allDaysDf = allDaysDf.append(thisDf)
      #  allDaysDf = allDaysDf.append(thisDf, ignore_index=True)

create  2021-02-18
append  2021-02-19
append  2021-02-22
append  2021-02-23
append  2021-03-07
append  2021-03-08
append  2021-03-09


In [5]:
allDaysDf.sort_values(by=['Client_Gender','Age_Group', 'Case_Year', 'Case_WeekNo', 'FileDate', 'Case_Count'], axis=0, inplace=True)

In [6]:
targetGroupDf = allDaysDf.drop_duplicates(subset=['Client_Gender', 'Age_Group', 'Case_Year', 'Case_WeekNo'], keep='first')

In [7]:
targetGroupDf['Changes'] = 'No'

<ipython-input-7-374fb5c2d9ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targetGroupDf['Changes'] = 'No'


In [8]:
targetGroupDf.head(20)

,Client_Gender,Age_Group,Case_Year,Case_WeekNo,Case_Count,FileDate,Changes
0,FEMALE,20s,2020,2,1,2021-02-18,No
1,FEMALE,20s,2020,4,2,2021-02-18,No
2,FEMALE,20s,2020,6,1,2021-02-18,No
3,FEMALE,20s,2020,8,2,2021-02-18,No
4,FEMALE,20s,2020,9,1,2021-02-18,No
5,FEMALE,20s,2020,10,12,2021-02-18,No
6,FEMALE,20s,2020,11,54,2021-02-18,No
7,FEMALE,20s,2020,12,167,2021-02-18,No
8,FEMALE,20s,2020,13,155,2021-02-18,No
9,FEMALE,20s,2020,14,163,2021-02-18,No


In [12]:
for index, row in targetGroupDf.iterrows():
    if ((allDaysDf.loc[(allDaysDf['Client_Gender'] == row['Client_Gender']) &
                 (allDaysDf['Age_Group'] == row['Age_Group']) &
                 (allDaysDf['Case_Year'] == row['Case_Year']) & 
                 (allDaysDf['Case_WeekNo'] == row['Case_WeekNo']) &
                 (allDaysDf['Case_Count'] != row['Case_Count'])]).shape[0] > 0):
        targetGroupDf.loc[index,'Changes'] = 'Yes'
        
        
        

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

...

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/rob/anaconda3/envs/covid19/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a c

In [15]:
targetGroupDf.loc[targetGroupDf['Changes'] == 'Yes']

,Client_Gender,Age_Group,Case_Year,Case_WeekNo,Case_Count,FileDate,Changes
4,FEMALE,20s,2020,9,1,2021-02-18,Yes
32,FEMALE,20s,2020,37,344,2021-02-18,Yes
35,FEMALE,20s,2020,40,596,2021-02-18,Yes
36,FEMALE,20s,2020,41,498,2021-02-18,Yes
40,FEMALE,20s,2020,45,983,2021-02-18,Yes
...,...,...,...,...,...,...,...
1327,UNSPECIFIED,<20,2021,8,28,2021-03-07,Yes
1328,UNSPECIFIED,<20,2021,9,21,2021-03-07,Yes
1286,UNSPECIFIED,UNKNOWN,2021,1,1,2021-02-18,Yes
1288,UNSPECIFIED,UNKNOWN,2021,7,2,2021-02-18,Yes
